<a href="https://colab.research.google.com/github/chandan9t8/UnivProjects/blob/main/Surface%20Crack%20Detection/SurfaceCrackDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports

import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop, Adagrad
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
import cv2
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pwd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip ./drive/MyDrive/Data/archive.zip > /dev/null

In [ ]:
# loading dataset (grayscale images)

labels = ['Negative', 'Positive']
img_size = 85
def read_images(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data, dtype=object)

Dataset = read_images('./')

In [ ]:
Dataset.shape

In [ ]:
Im = []
for i in Dataset:
    if(i[1] == 0):
        Im.append("Negative")
    elif(i[1] == 1):
        Im.append("Positive")

In [ ]:
# Visualising the dataset

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.hist(Im)
plt.title('Number of Images')
plt.xlabel('Label')

In [ ]:
# Normalizing gray scale Image Data
x = []
y = []

for feature, label in Dataset:
    x.append(feature)
    y.append(label)

x = np.array(x).reshape(-1, img_size, img_size, 1)
x = x / 255
y = np.array(y)

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(x[1000].reshape(img_size, img_size), cmap='gray')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(x[30000].reshape(img_size, img_size), cmap='gray')
plt.axis('off')

###CNN Model###

In [ ]:
# Model 1

model1 = Sequential(name="my_sequential1")
model1.add(Conv2D(64,3,padding="same", activation="relu", input_shape = x.shape[1:]))
model1.add(MaxPool2D())
model1.add(Conv2D(64, 3, padding="same", activation="relu"))
model1.add(MaxPool2D())
model1.add(Conv2D(128, 3, padding="same", activation="relu"))
model1.add(MaxPool2D())
model1.add(Flatten())
model1.add(Dense(256,activation="relu"))
model1.add(Dropout(0.5))
model1.add(BatchNormalization())
model1.add(Dense(2, activation="softmax"))
model1.summary()

In [ ]:
# Model 2

model2 = Sequential(name="my_sequential2")
model2.add(Conv2D(64,3,padding="same", activation="relu", input_shape = x.shape[1:]))
model2.add(MaxPool2D())
model2.add(Conv2D(64, 3, padding="same", activation="relu"))
model2.add(MaxPool2D())
model2.add(Conv2D(128, 3, padding="same", activation="relu"))
model2.add(MaxPool2D())
model2.add(Flatten())
model2.add(Dense(256,activation="relu"))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(2, activation="softmax"))
model2.summary()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
y_train.shape

In [ ]:
#Data Augmentation
datagen = ImageDataGenerator(featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=10,                                                      # randomly rotate images in the range(0 to 180)
                                  zoom_range = 0.1,                                                       # Randomly zoom image
                                  width_shift_range=0.1,                                                  # randomly shift images horizontally (fraction of total width)
                                  height_shift_range=0.1,                                                 # randomly shift images vertically (fraction of total height)
                                  horizontal_flip=False,
                                  vertical_flip=True)

datagen.fit(x_train)

In [ ]:
from keras.callbacks import ReduceLROnPlateau

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compile the model
model1.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

# Fit the model
epochs = 15
batch_size = 128
history1 = model1.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, verbose = 1, steps_per_epoch=len(x_train)//batch_size , callbacks=[learning_rate_reduction])

In [ ]:
# accuracy for model1

loss, accuracy = model1.evaluate(x_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
print(history1.history.keys())

In [ ]:
plt.figure(figsize=(12, 12))
plt.style.use('ggplot')
plt.subplot(2,2,1)
plt.plot(history1.history['accuracy'])
plt.title('Training accuracy of the Model')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Epoch', fontsize=12)

plt.subplot(2,2,2)
plt.plot(history1.history['loss'])
plt.title('Training loss of the Model')
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)

In [ ]:
opt = Adam(lr=1e-5)
model2.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
history2 = model2.fit(x_train, y_train, epochs = 15,validation_split = 0.25, batch_size = 128, verbose=1)

In [ ]:
# accuracy for model2
loss, accuracy = model2.evaluate(x_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
print(history2.history.keys())

In [ ]:
plt.figure(figsize=(12, 12))
plt.style.use('ggplot')
plt.subplot(2,2,1)
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('Accuracy of the Model')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(['train accuracy', 'validation accuracy'], loc='lower right', prop={'size': 12})

plt.subplot(2,2,2)
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Loss of the Model')
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(['train loss', 'validation loss'], loc='best', prop={'size': 12})

### Inception V3 ###

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
        if len(images)>3:
            break
    fig=plt.figure(figsize=(10,12))
    xrange=range(1,5)

    for img,x in zip(images,xrange):
        ax=fig.add_subplot(2,2,x)

        ax.imshow(img)
        ax.set_title(img.shape)

In [ ]:
# moving positive and negative examples into a single folder
!mkdir ./SurfaceCrack
!mv ./Negative ./Positive ./SurfaceCrack

In [ ]:
load_images_from_folder("./SurfaceCrack/Positive")

In [ ]:
load_images_from_folder("./SurfaceCrack/Negative")

In [ ]:
# Defining a Callback class that stops training once accuracy reaches 99.9%

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.999):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False, weights = 'imagenet')


for layer in pre_trained_model.layers:
     layer.trainable = False


In [ ]:
from keras import Model,layers

# using part of the pre-trained model from input layer to layer 'mixed7'
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)

# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout layer to prevent overfitting
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)

model3 = Model(pre_trained_model.input, x)
model3.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model3.summary()


In [ ]:
# Create Data Generator
# Split data train-validation sets by using validation_split=0.3
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.3,
                                  featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=10,                                                      # randomly rotate images in the range(0 to 180)
                                  zoom_range = 0.1,                                                       # Randomly zoom image
                                  width_shift_range=0.1,                                                  # randomly shift images horizontally (fraction of total width)
                                  height_shift_range=0.1,                                                 # randomly shift images vertically (fraction of total height)
                                  horizontal_flip=False,
                                  vertical_flip=True)

train_generator = train_datagen.flow_from_directory('./SurfaceCrack',
                                                     target_size=(150,150),
                                                     batch_size=64,
                                                     shuffle=True,
                                                     class_mode='binary',
                                                     subset='training')

In [ ]:
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                        validation_split=0.3,
                                        featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=10,                                                      # randomly rotate images in the range(0 to 180)
                                  zoom_range = 0.1,                                                       # Randomly zoom image
                                  width_shift_range=0.1,                                                  # randomly shift images horizontally (fraction of total width)
                                  height_shift_range=0.1,                                                 # randomly shift images vertically (fraction of total height)
                                  horizontal_flip=False,
                                  vertical_flip=True)


validation_generator =  validation_datagen.flow_from_directory('./SurfaceCrack',
                                                                target_size=(150,150),
                                                                batch_size=64,
                                                                class_mode='binary',
                                                                subset='validation')

In [ ]:
callbacks = myCallback()

history = model3.fit_generator(train_generator,
            validation_data = validation_generator,
            epochs = 7,
            verbose = 1,
            callbacks=[callbacks])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')

plt.legend()

plt.show()

### ResNet50 ###

In [ ]:
!pip install keras-resnet

In [ ]:
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
train_datagen = ImageDataGenerator(validation_split=0.3,
                                  featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=10,                                                      # randomly rotate images in the range(0 to 180)
                                  zoom_range = 0.1,                                                       # Randomly zoom image
                                  width_shift_range=0.1,                                                  # randomly shift images horizontally (fraction of total width)
                                  height_shift_range=0.1,                                                 # randomly shift images vertically (fraction of total height)
                                  horizontal_flip=False,
                                  vertical_flip=True,
                                  preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory('./SurfaceCrack',
                                                     target_size=(224,224),
                                                     batch_size=64,
                                                     shuffle=True,
                                                     class_mode='categorical',
                                                     subset='training')

validation_datagen = ImageDataGenerator(validation_split=0.3,
                                        featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=10,
                                  zoom_range = 0.1,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=True,
                                  preprocessing_function=preprocess_input)

validation_generator =  validation_datagen.flow_from_directory('./SurfaceCrack',
                                                                target_size=(224,224),
                                                                batch_size=64,
                                                                class_mode='categorical',
                                                                subset='validation')

In [ ]:
model_res50 = Sequential()

model_res50.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet'
    ))

model_res50.add(Dense(2, activation='softmax'))

model_res50.layers[0].trainable = False

model_res50.summary()

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

In [ ]:
callbacks = myCallback()

model_res50.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


fit_history = model_res50.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    validation_steps=steps_per_epoch_validation,
    epochs=7,
    validation_data=validation_generator,
    verbose=1,
    callbacks=[callbacks]
)

In [ ]:
acc = fit_history.history['accuracy']
val_acc = fit_history.history['val_accuracy']
loss = fit_history.history['loss']
val_loss = fit_history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')

plt.legend()

plt.show()

### VGG16 ###

In [ ]:
from tensorflow.keras.applications import VGG16


pretrained_model=VGG16(input_shape = (150, 150, 3),
                        include_top = False,
                        weights = 'imagenet')


for layer in pretrained_model.layers:
     layer.trainable = False

pretrained_model.summary()

In [ ]:
last_layer = pretrained_model.get_layer('block5_pool')
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model_vgg = Model(pretrained_model.input, x)


model_vgg.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.3,
                                  featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=30,                                                      # randomly rotate images in the range(0 to 180)
                                  zoom_range = 0.15,                                                      # Randomly zoom image
                                  width_shift_range=0.2,                                                  # randomly shift images horizontally (fraction of total width)
                                  height_shift_range=0.2,                                                 # randomly shift images vertically (fraction of total height)
                                  shear_range = 0.15,
                                  horizontal_flip=False,
                                  vertical_flip=True,)

train_generator = train_datagen.flow_from_directory('./SurfaceCrack',
                                                     target_size=(150,150),
                                                     batch_size=64,
                                                     shuffle=True,
                                                     class_mode='binary',
                                                     subset='training')

validation_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split=0.3,
                                  featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=10,                                                      # randomly rotate images in the range(0 to 180)
                                  zoom_range = 0.1,                                                       # Randomly zoom image
                                  width_shift_range=0.1,                                                  # randomly shift images horizontally (fraction of total width)
                                  height_shift_range=0.1,                                                 # randomly shift images vertically (fraction of total height)
                                  horizontal_flip=False,
                                  vertical_flip=True,
                                  preprocessing_function=preprocess_input)

validation_generator =  validation_datagen.flow_from_directory('./SurfaceCrack',
                                                                target_size=(150,150),
                                                                batch_size=64,
                                                                class_mode='binary',
                                                                subset='validation')

In [ ]:
callbacks = myCallback()

history = model_vgg.fit_generator(train_generator,
                                  validation_data = validation_generator,
                                  epochs = 7,
                                  verbose = 1,
                                  callbacks=[callbacks])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')

plt.legend()

plt.show()